## Problema de la Mochila

(Knapdac

### Solución cuántica a un problema clásico CQM con reformulación QUBO.

V.1, Julio 2023

### El Problema de la mochila clásico

En el problema de mochila, se debe de empaquetar un conjunto de elementos de peso (o volumen) determinado en una mochila con una peso (o volumen) máximo. 

Además, estos elemenos tienen diferente valor para el mochilero, bien por su utilidad, bien por su coste

Si el tamaño o peso total de los elementos supera lo soportado por la mochila, no se pueden empaquetar todos. En ese caso, el problema es elegir un subconjunto de los elementos  que cabiendo en la mochila maximicen el valor de la carga.

A continuación se va a especificar un problema ejemplo, con 5 elementos como objetivo

<b> Parámetrización del problema</b>

    - Objetos:  N = 5 

    - Peso_max: w_max = 15
    
    - Pesos i : wi={1, 2, 1, 2, 2 }

    - Utilidad (1-5): ui={1, 3, 2, 5, 4}

Función objetivo: maximizar la utilidad asociada a los objetos en la mochila sujetos a la restricción del peso max

<b>Nota<b>
    
En los problemas CQM/BQM la función objetivo siempre se minimiza, de ahí los signos (-) en cqm.set_objective 

In [41]:
# Recursos
import dimod
from dimod import ConstrainedQuadraticModel, ExactCQMSolver, Integer, Binary
from dwave.system import DWaveSampler, EmbeddingComposite

# Funciones auxiliares

def sol_factibles(sampleset):
    # Resultados
    print("\nLas soluciones factibles al problema son:\n")
    print(sampleset.filter(lambda s: s.is_feasible).aggregate())
    
def sol_problema(result):
    # Agregación de los n reads
    samples = []
    ocurrencias = []

    for s in result.data():
        samples.append(invert(s.sample))
        ocurrencias.append(s.num_occurrences)

    sampleset = dimod.SampleSet.from_samples_cqm(samples,cqm,num_occurrences=ocurrencias)
    return(sampleset)


In [42]:
# Problema mochila como problema CQM de variable discreta

N=5
w=6
wi=[1,2,1,3,2]
ui=[1,3,2,5,4]
 
#Uso de QP

sampler = EmbeddingComposite(DWaveSampler())

# dimod.Integer impone que upper_bound >=2

rango= lambda s: int(w/s) if int(w/s) > 2 else 2 

x1 = dimod.Integer('x1', upper_bound=rango(wi[0]))
x2 = dimod.Integer('x2', upper_bound=rango(wi[1]))
x3 = dimod.Integer('x3', upper_bound=rango(wi[2]))
x4 = dimod.Integer('x4', upper_bound=rango(wi[3]))
x5 = dimod.Integer('x5', upper_bound=rango(wi[4]))


cqm = dimod.ConstrainedQuadraticModel()

cqm.set_objective(-ui[0]*x1-ui[1]*x2-ui[2]*x3-ui[3]*x4-ui[4]*x5)

cqm.add_constraint(wi[0]*x1+wi[1]*x2+wi[2]*x3+wi[3]*x4+wi[4]*x5 <= w, label='restricción peso max')


'restricción peso max'

In [43]:
print("Variables:")
print(cqm.variables)
print("Objetivo:")
print(cqm.objective)
print("Restricciones:") 
print(cqm.constraints)

Variables:
Variables(['x1', 'x2', 'x3', 'x4', 'x5'])
Objetivo:
ObjectiveView({'x1': -1.0, 'x2': -3.0, 'x3': -2.0, 'x4': -5.0, 'x5': -4.0}, {}, 0.0, {'x1': 'INTEGER', 'x2': 'INTEGER', 'x3': 'INTEGER', 'x4': 'INTEGER', 'x5': 'INTEGER'})
Restricciones:
{'restricción peso max': Le(ConstraintView({'x1': 1.0, 'x2': 2.0, 'x3': 1.0, 'x4': 3.0, 'x5': 2.0}, {}, 0.0, {'x1': 'INTEGER', 'x2': 'INTEGER', 'x3': 'INTEGER', 'x4': 'INTEGER', 'x5': 'INTEGER'}), 6.0)}


In [44]:
#Reformulación como problema qubo

fl=10

qubo, invert = dimod.cqm_to_bqm(cqm, lagrange_multiplier = fl)
result = sampler.sample(qubo, num_reads=100)

In [45]:
sampleset=sol_problema(result)
sol_factibles(sampleset)


Las soluciones factibles al problema son:

   x1 x2 x3 x4 x5 energy num_oc. is_sat. is_fea.
0   0  0  6  0  0  -12.0       1 arra...    True
1   1  0  5  0  0  -11.0       1 arra...    True
2   0  0  1  1  1  -11.0       3 arra...    True
3   0  0  3  1  0  -11.0       1 arra...    True
4   1  0  0  1  1  -10.0       4 arra...    True
5   0  1  1  1  0  -10.0       1 arra...    True
6   2  0  2  0  1  -10.0       1 arra...    True
22  0  0  3  0  1  -10.0       2 arra...    True
23  1  1  3  0  0  -10.0       1 arra...    True
34  0  0  0  2  0  -10.0       1 arra...    True
7   1  0  4  0  0   -9.0       1 arra...    True
8   0  0  2  1  0   -9.0       3 arra...    True
9   2  0  1  1  0   -9.0       2 arra...    True
24  0  1  3  0  0   -9.0       2 arra...    True
25  1  0  2  0  1   -9.0       1 arra...    True
26  0  0  0  1  1   -9.0       2 arra...    True
38  2  1  0  0  1   -9.0       1 arra...    True
10  0  0  2  0  1   -8.0       1 arra...    True
11  2  0  3  0  0   -8.0 


<b>Comentarios</b>

- La solución óptima más diversa es x3,x5=2 con utilidad acumulada =12
- La solución cuasi óptima es x3,x4,x5=1, con utilidad total=11 

<b>nota:</b>

Con fl>>10 se producen ausencias significativas, p.e.

x1 x2 x3 x4 x5=[0 1 0 0 2], f=-11 no es evaluada como solución candidata.

### Mejoras

Es trivial ampliar el problema para que contemple restricción de peso y volumen.